<a href="https://colab.research.google.com/github/matthew-e-thomas/Plant_Image_Project/blob/main/model_update_echo_group_module8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Libraries, get paths ready, and clone PlantDoc Repo

In [ ]:
import numpy as np
import tensorflow as tf
import IPython.display as display
print(tf.__version__)

2.3.0


In [ ]:
!python --version

Python 3.6.9


In [ ]:
!git clone https://github.com/pratikkayal/PlantDoc-Dataset.git

Cloning into 'PlantDoc-Dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 2658 (delta 17), reused 0 (delta 0), pack-reused 2628
Receiving objects: 100% (2658/2658), 932.25 MiB | 44.72 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Checking out files: 100% (2580/2580), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#run this if cloning plantdoc github
from pathlib import Path
data_dir = Path.cwd() / 'PlantDoc-Dataset'

In [ ]:
train_dir = data_dir.joinpath('train')
test_dir = data_dir.joinpath('test')

In [ ]:
#CLASS_NAMES_TRAIN = [item.name for item in train_dir.glob('*')]

In [ ]:
#run this if using improved plantdoc dataset from your google drive
# import os
# os.chdir('/content/drive/My Drive')

In [ ]:
#!unzip PlantDoc.v1-resize-416x416.tfrecord.zip

Archive:  PlantDoc.v1-resize-416x416.tfrecord.zip
 extracting: train/leaves.tfrecord   
 extracting: train/leaves_label_map.pbtxt  
 extracting: test/leaves.tfrecord    
 extracting: test/leaves_label_map.pbtxt  
 extracting: README.roboflow.txt     
 extracting: README.dataset.txt      


In [ ]:
filepaths = ["train/leaves.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset.take(5):
    print(item)

tf.Tensor(b'\n\x9e\x96\x02\n\xf3\x92\x02\n\rimage/encoded\x12\xe0\x92\x02\n\xdc\x92\x02\n\xd8\x92\x02\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01\xa0\x01\xa0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\x

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset(['train/leaves.tfrecord'])

# Create a dictionary describing the features.
image_feature_description = {
    #'height': tf.io.FixedLenFeature([], tf.int64),
    #'width': tf.io.FixedLenFeature([], tf.int64),
    #'depth': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {label: ()}, types: {label: tf.string}>

In [ ]:
for image_features in parsed_image_dataset.take(5):
  image_raw = image_features['image'].numpy()
  display.display(display.Image(data=image_raw))

InvalidArgumentError: ignored

### Create Train, Validation, and Test Set

In [ ]:
batch_size = 32
# img_height = 150
# img_width = 150
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  #label_mode="categorical",
  #class_names=CLASS_NAMES,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2342 files belonging to 28 classes.
Using 2108 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  #label_mode="categorical",
  #class_names=CLASS_NAMES,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2342 files belonging to 28 classes.
Using 234 files for validation.


In [ ]:
# class names
class_names = train_ds.class_names
print(class_names)
print(len(class_names))

['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot']
28


In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=123,
  #label_mode="categorical",
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 236 files belonging to 27 classes.


### Create and Train Model

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications import inception_resnet_v2

In [ ]:
from keras.applications import InceptionResNetV2

In [ ]:
def preprocess(image, label):
    #resized_image = tf.image.resize(image, [224, 224])
    final_image = inception_resnet_v2.preprocess_input(image)
    return final_image, label

In [ ]:

train_set = train_ds.map(preprocess).prefetch(1)
valid_set = val_ds.map(preprocess).prefetch(1)
test_set = test_ds.map(preprocess).prefetch(1)

In [ ]:
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential

# # data augmentation
# data_augmentation = keras.Sequential(
#   [
#     layers.experimental.preprocessing.RandomFlip("horizontal", 
#                                                  input_shape=(img_height, 
#                                                               img_width,
#                                                               3)),
#     layers.experimental.preprocessing.RandomRotation(0.1),
#     layers.experimental.preprocessing.RandomZoom(0.1),
#   ]
# )



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

monitor_val_acc = EarlyStopping(monitor = 'val_loss', patience=15)

In [ ]:
tf.random.set_seed(123)
np.random.seed(123)
base_model = InceptionResNetV2(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
#x = keras.layers.Dense(1024, activation='relu')(avg)
output = keras.layers.Dense(len(class_names), activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

219062272/219055592 [==============================] - 1s 0us/step


In [ ]:
# for layer in base_model.layers:
#     layer.trainable = False

In [ ]:
adagrad = tf.keras.optimizers.Adagrad()
nadam = tf.keras.optimizers.Nadam(.0001)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=nadam, metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=40, callbacks=monitor_val_acc)
score = model.evaluate(test_set)

Epoch 1/40
66/66 [==============================] - 50s 756ms/step - loss: 2.3668 - accuracy: 0.3235 - val_loss: 1.6350 - val_accuracy: 0.5214
Epoch 2/40
66/66 [==============================] - 46s 702ms/step - loss: 0.8575 - accuracy: 0.7685 - val_loss: 1.3983 - val_accuracy: 0.5299
Epoch 3/40
66/66 [==============================] - 46s 697ms/step - loss: 0.2696 - accuracy: 0.9379 - val_loss: 1.4449 - val_accuracy: 0.5556
Epoch 4/40
66/66 [==============================] - 46s 697ms/step - loss: 0.1111 - accuracy: 0.9715 - val_loss: 1.5128 - val_accuracy: 0.6026
Epoch 5/40
66/66 [==============================] - 46s 697ms/step - loss: 0.0783 - accuracy: 0.9758 - val_loss: 1.6187 - val_accuracy: 0.5684
Epoch 6/40
66/66 [==============================] - 46s 697ms/step - loss: 0.0618 - accuracy: 0.9791 - val_loss: 1.6537 - val_accuracy: 0.6026
Epoch 7/40
66/66 [==============================] - 46s 693ms/step - loss: 0.0502 - accuracy: 0.9839 - val_loss: 1.6042 - val_accuracy: 0.5897

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_6
1 conv2d_1015
2 batch_normalization_1015
3 activation_1015
4 conv2d_1016
5 batch_normalization_1016
6 activation_1016
7 conv2d_1017
8 batch_normalization_1017
9 activation_1017
10 max_pooling2d_20
11 conv2d_1018
12 batch_normalization_1018
13 activation_1018
14 conv2d_1019
15 batch_normalization_1019
16 activation_1019
17 max_pooling2d_21
18 conv2d_1023
19 batch_normalization_1023
20 activation_1023
21 conv2d_1021
22 conv2d_1024
23 batch_normalization_1021
24 batch_normalization_1024
25 activation_1021
26 activation_1024
27 average_pooling2d_5
28 conv2d_1020
29 conv2d_1022
30 conv2d_1025
31 conv2d_1026
32 batch_normalization_1020
33 batch_normalization_1022
34 batch_normalization_1025
35 batch_normalization_1026
36 activation_1020
37 activation_1022
38 activation_1025
39 activation_1026
40 mixed_5b
41 conv2d_1030
42 batch_normalization_1030
43 activation_1030
44 conv2d_1028
45 conv2d_1031
46 batch_normalization_1028
47 batch_normalization_1031
48 activation_1028
49 activation

In [ ]:
for layer in model.layers[:673]:
   layer.trainable = False
for layer in model.layers[673:]:
   layer.trainable = True

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=nadam, metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=35, callbacks=monitor_val_acc)
score = model.evaluate(test_set)

Epoch 1/35
66/66 [==============================] - 33s 505ms/step - loss: 0.0352 - accuracy: 0.9891 - val_loss: 1.7143 - val_accuracy: 0.6111
Epoch 2/35
66/66 [==============================] - 30s 461ms/step - loss: 0.0143 - accuracy: 0.9938 - val_loss: 1.7437 - val_accuracy: 0.6111
Epoch 3/35
66/66 [==============================] - 30s 457ms/step - loss: 0.0127 - accuracy: 0.9948 - val_loss: 1.7636 - val_accuracy: 0.6154
Epoch 4/35
66/66 [==============================] - 31s 465ms/step - loss: 0.0125 - accuracy: 0.9938 - val_loss: 1.7508 - val_accuracy: 0.6197
Epoch 5/35
66/66 [==============================] - 30s 458ms/step - loss: 0.0099 - accuracy: 0.9948 - val_loss: 1.7423 - val_accuracy: 0.6239
Epoch 6/35
66/66 [==============================] - 31s 463ms/step - loss: 0.0103 - accuracy: 0.9943 - val_loss: 1.7732 - val_accuracy: 0.6282
Epoch 7/35
66/66 [==============================] - 30s 459ms/step - loss: 0.0101 - accuracy: 0.9938 - val_loss: 1.8112 - val_accuracy: 0.6197

In [ ]:
#model.save('/content/drive/My Drive/plant_inceptionresnet_model')
model.save('/content/drive/My Drive/plant_inceptionresnet_model_h5.h5')

### Xception Model

In [ ]:
from keras.applications import Xception

In [ ]:
base_model = Xception(include_top=False, 
                               weights='imagenet', 
                               pooling='avg')

output = keras.layers.Dense(len(class_names), activation='softmax')(base_model.output)
model = keras.models.Model(inputs=base_model.inputs, outputs=output)
model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer="nadam", 
    metrics=["accuracy"])
history = model.fit(train_ds, epochs=10)
score = model.evaluate(test_ds)

Epoch 1/10
59/59 [==============================] - 43s 721ms/step - loss: 2.1950 - accuracy: 0.3570
Epoch 2/10
59/59 [==============================] - 42s 716ms/step - loss: 1.0706 - accuracy: 0.6601
Epoch 3/10
59/59 [==============================] - 43s 727ms/step - loss: 0.6377 - accuracy: 0.8063
Epoch 4/10
59/59 [==============================] - 43s 731ms/step - loss: 0.3776 - accuracy: 0.8895
Epoch 5/10
59/59 [==============================] - 42s 715ms/step - loss: 0.3023 - accuracy: 0.9088
Epoch 6/10
59/59 [==============================] - 43s 734ms/step - loss: 0.3845 - accuracy: 0.8847
Epoch 7/10
59/59 [==============================] - 42s 718ms/step - loss: 0.2582 - accuracy: 0.9146
Epoch 8/10
59/59 [==============================] - 42s 708ms/step - loss: 0.2441 - accuracy: 0.9216
Epoch 9/10
59/59 [==============================] - 42s 713ms/step - loss: 0.1586 - accuracy: 0.9472
Epoch 10/10
8/8 [==============================] - 1s 109ms/step - loss: 4.0204 - accuracy: